In [ ]:
import cv2
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

%matplotlib inline


In [ ]:
imgSize = (1280, 720)

calChessboardPattern = (4, 4)
calImgsPath = glob.glob("./ccalib-test-images-3/*.jpg")

objPoints = []
imgPoints = []

objPt = np.zeros((calChessboardPattern[0] * calChessboardPattern[1], 3), np.float32)
objPt[:, :2] = np.mgrid[0:calChessboardPattern[0], 0:calChessboardPattern[1]].T.reshape(-1, 2)

fig = plt.figure(figsize=(16, 14))
fig.suptitle("Part of sample chessboard images for camera calibration", fontsize=32)
columns = 3
rows = 6
i = 1

for image in calImgsPath:
    calImg = cv2.imread(image)
    ret, corners = cv2.findChessboardCorners(calImg, calChessboardPattern)
    
    if ret:
        imgPoints.append(corners)
        objPoints.append(objPt)
        cv2.drawChessboardCorners(calImg, calChessboardPattern, corners, ret)
        if i <= rows * columns:
            sub_i = fig.add_subplot(rows, columns, i)
            sub_i.set_title("cos")
            plt.imshow(calImg)
            i += 1
        else:
            break
plt.show()

In [ ]:
retval, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(objPoints, imgPoints, imgSize, None, None)

In [ ]:
print("Camera matrix: \n", cameraMatrix)
print("\nDistortion coefficients: \n", distCoeffs)

In [ ]:
optimAlpha = 1
optimCameraMatrix, cameraROI = cv2.getOptimalNewCameraMatrix(cameraMatrix, distCoeffs, imgSize, optimAlpha)
print("Optimal camera matrix: \n", optimCameraMatrix)
print("\nRegion of interest:\n", cameraROI)

In [ ]:
def undistortImage(img):
    imgUndistRaw = imgUndist = cv2.undistort(img, cameraMatrix, distCoeffs, None, optimCameraMatrix)
    x, y, w, h = cameraROI
    x, y, w, h = int(x), int(y), int(w), int(h)
    imgUndistCrop = imgUndist[y:y + h, x:x + w]
    imgUndistScaled = cv2.resize(imgUndistCrop, imgSize)
    return imgUndistScaled, imgUndistCrop, imgUndistRaw

In [ ]:
testImg = cv2.imread("./samples/double_chessboard_1.jpg")
testUndistScaled, testUndistCrop, testUndistRaw = undistortImage(testImg)
fig = plt.figure(figsize=(16, 8))

sub1 = fig.add_subplot(2, 2, 1)
sub1.set_title("Original image")
plt.imshow(testImg)

sub2 = fig.add_subplot(2, 2, 2)
sub2.set_title("Raw undistorted image")
plt.imshow(testUndistRaw)

sub3 = fig.add_subplot(2, 2, 3)
sub3.set_title("Undistorted cropped to ROI image")
plt.imshow(testUndistCrop)
           
sub4 = fig.add_subplot(2, 2, 4)
sub4.set_title("Undistorted scaled image")
plt.imshow(testUndistScaled)
cv2.imwrite("./samples/double_chessboard_1_result.jpg", testUndistScaled)

In [ ]:
sampleRoadImgs = glob.glob("./samples/road_samples_1/raw/*.png")

fig = plt.figure(figsize=(16, 16))
fig.suptitle("Part of sample chessboard images for camera calibration", fontsize=32)
columns = 2
rows = 4
i = 1

for image in sampleRoadImgs:
    img = cv2.imread(image)
    imgUndist, _, _ = undistortImage(img)
    cv2.imwrite("./samples/road_samples_1/undistorted/"+os.path.basename(image).replace(".png", "_undist.png"), imgUndist)
    if i <= columns * rows:
        sub_i = fig.add_subplot(rows, columns, i).axis('off')
        plt.imshow(img)
        i += 1
        sub_i = fig.add_subplot(rows, columns, i).axis('off')
        plt.imshow(imgUndist)
        i += 1
plt.show()

In [ ]:
srcWarpPoints = np.float32([[23, imgSize[1]], [1216, imgSize[1]],
                        [757, imgSize[1] * 0.55], [527, imgSize[1] * 0.55]])

dstWarpPoints = np.float32([[imgSize[0] / 4, imgSize[1]],
                         [imgSize[0] * 3 / 4, imgSize[1]],
                         [imgSize[0] * 3 / 4, 0],
                         [imgSize[0] / 4, 0]])

warpMatrix = cv2.getPerspectiveTransform(srcWarpPoints, dstWarpPoints)

def drawWarpLines(img, imgWarped):
    for i in range(4):
        cv2.line(img, tuple(srcWarpPoints[i]), tuple(srcWarpPoints[(i + 1) % 4]), (255, 0, 0), thickness=10)
        cv2.line(imgWarped, tuple(dstWarpPoints[i]), tuple(dstWarpPoints[(i + 1) % 4]), (255, 0, 0), thickness=10)
    return img, imgWarped

blackOrg = np.zeros(imgSize[::-1])
blackWarp = np.zeros(imgSize[::-1])
blackOrg, blackWarp = drawWarpLines(blackOrg, blackWarp)

fig = plt.figure(figsize=(16, 8))
fig.add_subplot(1, 2, 1).axis('off')
plt.imshow(blackOrg)
fig.add_subplot(1, 2, 2).axis('off')
plt.imshow(blackWarp)

In [ ]:
def warpView(img):
    imgWarped = cv2.warpPerspective(img, warpMatrix, imgSize)
    return imgWarped

In [ ]:
sampleRoadImgs = glob.glob("./samples/road_samples_1/undistorted/*.png")

fig = plt.figure(figsize=(16, 16))
columns = 2
rows = 6
i = 1

for image in sampleRoadImgs:
    img = cv2.imread(image)
    imgWarped = warpView(img)
    cv2.imwrite("./samples/road_samples_1/warped/"+os.path.basename(image).replace("_undist.png", "_warped.png"), imgWarped)
    if i <= columns * rows:
        img, imgWarped = drawWarpLines(img, imgWarped)
        sub_i = fig.add_subplot(rows, columns, i).axis('off')
        plt.imshow(img)
        i += 1
        sub_i = fig.add_subplot(rows, columns, i).axis('off')
        plt.imshow(imgWarped)
        i += 1
plt.show()

In [ ]:
# sampleRoadImgs = glob.glob("./samples/road_samples_1/warped/*.png")

# for image in sampleRoadImgs:
fig = plt.figure(figsize=(16, 16))
columns = 3
rows = 5
# imgRGB = cv2.imread(image)
# imgBGR = cv2.imread("./samples/road_samples_1/warped/yellow_white_sample_warped.png")
imgBGR = cv2.imread("./samples/road_samples_1/warped/sample_road_img_6_warped.png")
imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)

imgRGB_R, imgRGB_G, imgRGB_B = cv2.split(imgBGR)

imgHSV = cv2.cvtColor(imgRGB, cv2.COLOR_RGB2HSV)
imgHSV_H, imgHSV_S, imgHSV_V = cv2.split(imgHSV)

imgHLS = cv2.cvtColor(imgRGB, cv2.COLOR_RGB2HLS)
imgHLS_H, imgHLS_L, imgHLS_S = cv2.split(imgHLS)

imgLUV = cv2.cvtColor(imgRGB, cv2.COLOR_RGB2LUV)
imgLUV_L, imgLUV_U, imgLUV_V = cv2.split(imgLUV)

imgLAB = cv2.cvtColor(imgRGB, cv2.COLOR_RGB2LAB)
imgLAB_L, imgLAB_A, imgLAB_B = cv2.split(imgLAB)

subRGB_R = fig.add_subplot(rows, columns, 1)
subRGB_R.axis('off')
subRGB_R.set_title('R')
plt.imshow(imgRGB_R, cmap='gray')
subRGB_G = fig.add_subplot(rows, columns, 2)
subRGB_G.axis('off')
subRGB_G.set_title('G')
plt.imshow(imgRGB_G, cmap='gray')
subRGB_B = fig.add_subplot(rows, columns, 3)
subRGB_B.axis('off')
subRGB_B.set_title('B')
plt.imshow(imgRGB_B, cmap='gray')

subHSV_H = fig.add_subplot(rows, columns, 4)
subHSV_H.axis('off')
subHSV_H.set_title('H')
plt.imshow(imgHSV_H, cmap='gray')
subHSV_S = fig.add_subplot(rows, columns, 5)
subHSV_S.axis('off')
subHSV_S.set_title('S')
plt.imshow(imgHSV_S, cmap='gray')
subHSV_V = fig.add_subplot(rows, columns, 6)
subHSV_V.axis('off')
subHSV_V.set_title('V')
plt.imshow(imgHSV_V, cmap='gray')

subHLS_H = fig.add_subplot(rows, columns, 7)
subHLS_H.axis('off')
subHLS_H.set_title('H')
plt.imshow(imgHLS_H, cmap='gray')
subHLS_L = fig.add_subplot(rows, columns, 8)
subHLS_L.axis('off')
subHLS_L.set_title('L')
plt.imshow(imgHLS_L, cmap='gray')
subHLS_S = fig.add_subplot(rows, columns, 9)
subHLS_S.axis('off')
subHLS_S.set_title('S')
plt.imshow(imgHLS_S, cmap='gray')

subLAB_L = fig.add_subplot(rows, columns, 10)
subLAB_L.axis('off')
subLAB_L.set_title('L')
plt.imshow(imgLAB_L, cmap='gray')
subLAB_A = fig.add_subplot(rows, columns, 11)
subLAB_A.axis('off')
subLAB_A.set_title('A')
plt.imshow(imgLAB_A, cmap='gray')
subLAB_B = fig.add_subplot(rows, columns, 12)
subLAB_B.axis('off')
subLAB_B.set_title('B')
plt.imshow(imgLAB_B, cmap='gray')

subLUV_L = fig.add_subplot(rows, columns, 13)
subLUV_L.axis('off')
subLUV_L.set_title('L')
plt.imshow(imgLUV_L, cmap='gray')
subLUV_U = fig.add_subplot(rows, columns, 14)
subLUV_U.axis('off')
subLUV_U.set_title('U')
plt.imshow(imgLUV_U, cmap='gray')
subLUV_V = fig.add_subplot(rows, columns, 15)
subLUV_V.axis('off')
subLUV_V.set_title('V')
plt.imshow(imgLUV_V, cmap='gray')

plt.show()

In [ ]:
def lowThreshFun(wHigh, img, lowThresh, title):
    if wHigh.result is not None:
        _, imgThresh = cv2.threshold(img, lowThresh, wHigh.result, cv2.THRESH_BINARY)
        fig = plt.figure()
        subP = fig.add_subplot(1, 1, 1)
        subP.axis('off')
        subP.set_title(title)
        plt.imshow(imgThresh)

def setHighT_RGB_R(highThresh):
    return highThresh

def setLowT_RGB_R(lowThresh):
    lowThreshFun(wRGB_R_high, imgRGB_R, lowThresh, 'RGB_R')
    return lowThresh

def setHighT_RGB_G(highThresh):
    return highThresh

def setLowT_RGB_G(lowThresh):
    lowThreshFun(wRGB_G_high, imgRGB_G, lowThresh, 'RGB_G')
    return lowThresh

def setHighT_RGB_B(highThresh):
    return highThresh

def setLowT_RGB_B(lowThresh):
    lowThreshFun(wRGB_B_high, imgRGB_B, lowThresh, 'RGB_B')
    return lowThresh

def setHighT_HSV_S(highThresh):
    return highThresh

def setLowT_HSV_S(lowThresh):
    lowThreshFun(wHSV_S_high, imgHSV_S, lowThresh, 'HSV_S')
    return lowThresh

def setHighT_HSV_V(highThresh):
    return highThresh

def setLowT_HSV_V(lowThresh):
    lowThreshFun(wHSV_V_high, imgHSV_V, lowThresh, 'HSV_V')
    return lowThresh

def setHighT_HLS_L(highThresh):
    return highThresh

def setLowT_HLS_L(lowThresh):
    lowThreshFun(wHLS_L_high, imgHLS_L, lowThresh, 'HLS_L')
    return lowThresh

def setHighT_HLS_S(highThresh):
    return highThresh

def setLowT_HLS_S(lowThresh):
    lowThreshFun(wHLS_S_high, imgHLS_S, lowThresh, 'HLS_S')
    return lowThresh

def setHighT_LAB_L(highThresh):
    return highThresh

def setLowT_LAB_L(lowThresh):
    lowThreshFun(wLAB_L_high, imgLAB_L, lowThresh, 'LAB_L')
    return lowThresh

def setHighT_LAB_B(highThresh):
    return highThresh

def setLowT_LAB_B(lowThresh):
    lowThreshFun(wLAB_B_high, imgLAB_B, lowThresh, 'LAB_B')
    return lowThresh

def setHighT_LUV_L(highThresh):
    return highThresh

def setLowT_LUV_L(lowThresh):
    lowThreshFun(wLUV_L_high, imgLUV_L, lowThresh, 'LUV_L')
    return lowThresh

def setHighT_LUV_U(highThresh):
    return highThresh

def setLowT_LUV_U(lowThresh):
    lowThreshFun(wLUV_U_high, imgLUV_U, lowThresh, 'LUV_U')
    return lowThresh

def setHighT_LUV_V(highThresh):
    return highThresh

def setLowT_LUV_V(lowThresh):
    lowThreshFun(wLUV_V_high, imgLUV_V, lowThresh, 'LUV_V')
    return lowThresh


wRGB_R_high = interactive(setHighT_RGB_R, highThresh=widgets.IntSlider(min=0,max=255,step=1,value=255))
wRGB_R_low = interactive(setLowT_RGB_R, lowThresh=widgets.IntSlider(min=0,max=255,step=1,value=135))
display(wRGB_R_high, wRGB_R_low)

wRGB_G_high = interactive(setHighT_RGB_G, highThresh=widgets.IntSlider(min=0,max=255,step=1,value=255))
wRGB_G_low = interactive(setLowT_RGB_G, lowThresh=widgets.IntSlider(min=0,max=255,step=1,value=170))
display(wRGB_G_high, wRGB_G_low)

wRGB_B_high = interactive(setHighT_RGB_B, highThresh=widgets.IntSlider(min=0,max=255,step=1,value=255))
wRGB_B_low = interactive(setLowT_RGB_B, lowThresh=widgets.IntSlider(min=0,max=255,step=1,value=180))
display(wRGB_B_high, wRGB_B_low)

wHSV_S_high = interactive(setHighT_HSV_S, highThresh=widgets.IntSlider(min=0,max=255,step=1,value=255))
wHSV_S_low = interactive(setLowT_HSV_S, lowThresh=widgets.IntSlider(min=0,max=255,step=1,value=145))
display(wHSV_S_high, wHSV_S_low)

wHSV_V_high = interactive(setHighT_HSV_V, highThresh=widgets.IntSlider(min=0,max=255,step=1,value=255))
wHSV_V_low = interactive(setLowT_HSV_V, lowThresh=widgets.IntSlider(min=0,max=255,step=1,value=145))
display(wHSV_V_high, wHSV_V_low)

wHLS_L_high = interactive(setHighT_HLS_L, highThresh=widgets.IntSlider(min=0,max=255,step=1,value=255))
wHLS_L_low = interactive(setLowT_HLS_L, lowThresh=widgets.IntSlider(min=0,max=255,step=1,value=135))
display(wHLS_L_high, wHLS_L_low)

wHLS_S_high = interactive(setHighT_HLS_S, highThresh=widgets.IntSlider(min=0,max=255,step=1,value=255))
wHLS_S_low = interactive(setLowT_HLS_S, lowThresh=widgets.IntSlider(min=0,max=255,step=1,value=115))
display(wHLS_S_high, wHLS_S_low)

wLAB_L_high = interactive(setHighT_LAB_L, highThresh=widgets.IntSlider(min=0,max=255,step=1,value=255))
wLAB_L_low = interactive(setLowT_LAB_L, lowThresh=widgets.IntSlider(min=0,max=255,step=1,value=145))
display(wLAB_L_high, wLAB_L_low)

wLAB_B_high = interactive(setHighT_LAB_B, highThresh=widgets.IntSlider(min=0,max=255,step=1,value=255))
wLAB_B_low = interactive(setLowT_LAB_B, lowThresh=widgets.IntSlider(min=0,max=255,step=1,value=155))
display(wLAB_B_high, wLAB_B_low)

wLUV_L_high = interactive(setHighT_LUV_L, highThresh=widgets.IntSlider(min=0,max=255,step=1,value=255))
wLUV_L_low = interactive(setLowT_LUV_L, lowThresh=widgets.IntSlider(min=0,max=255,step=1,value=173))
display(wLUV_L_high, wLUV_L_low)

wLUV_U_high = interactive(setHighT_LUV_U, highThresh=widgets.IntSlider(min=0,max=255,step=1,value=255))
wLUV_U_low = interactive(setLowT_LUV_U, lowThresh=widgets.IntSlider(min=0,max=255,step=1,value=112))
display(wLUV_U_high, wLUV_U_low)

wLUV_V_high = interactive(setHighT_LUV_V, highThresh=widgets.IntSlider(min=0,max=255,step=1,value=255))
wLUV_V_low = interactive(setLowT_LUV_V, lowThresh=widgets.IntSlider(min=0,max=255,step=1,value=170))
display(wLUV_V_high, wLUV_V_low)
